## Connecting to Neo4j db and  query

In [58]:
from dotenv import load_dotenv
import os
load_dotenv()
password=os.getenv('password')
user=os.getenv('user')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
Google_API_KEY=os.getenv('Google_api_key')

In [42]:
from neo4j import GraphDatabase
from py2neo import Graph
import pandas as pd
import neo4j

uri = "bolt://localhost:7687"
user = user
password = password
#database_name = "SchoolGPT.SchoolAI_dbms"  # Specify your database name
# add db name to avoid starting theough neo4j desktop
driver = GraphDatabase.driver(uri, auth=(user, password))


In [43]:
query="MATCH (n)-[r]->(m) RETURN n, r, m LIMIT 10"
            #"MATCH (n1:Node {name: 'system'}) RETURN n1 as node_name" #write proper queries
with driver.session() as session:
    result = session.run(query)
    n = [record['m'] for record in result]

### Displaying cypher query responses as pdf

In [1]:
def run_query_and_return_df(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())


In [118]:
query1 = """
MATCH (c1:Concept)
WHERE c1.concept_1 CONTAINS 'internal energy' AND c1.edge IS NOT NULL
RETURN c1;
"""
query2="""MATCH (n)-[r]->(m) RETURN n.name as concept_1, r.edge as r_edge, r.description as r_description, m.name as  concept_2"""

query3="""MATCH (n)-[r]->(m)
WHERE n.name CONTAINS 'internal energy' AND r.edge contains 'related'
RETURN n.name as concept_1,
r.edge as r_edge, r.description as r_description, m.name as concept_2"""

In [119]:
df2=run_query_and_return_df(query2)
df3=run_query_and_return_df(query3)

In [112]:
df3.shape

(31, 4)

In [120]:
df3

,concept_1,r_edge,r_description,concept_2
0,internal energy,is-related-to,If we neglect small intermolecular forces in a...,sum of kinetic energies
1,internal energy,is-related-to,The internal energy of a gas is the sum of the...,potential energy
2,internal energy,is-related-to,Internal energy is a concept in classical ther...,thermodynamic system
3,change in internal energy,is related to,the value of change in internal energy depends...,initial state
4,change in internal energy,is related to,represents the amount of heat transferred to o...,q
5,change in internal energy,is related to,the value of change in internal energy depends...,final state
6,change in internal energy,is related to,represents the amount of work done on or by a ...,w


In [78]:
## This function is not working can be ignored
pandas_df = driver.execute_query(
    """ MATCH (n)-[r]->(m) RETURN n, r, m LIMIT 10""",
    result_transformer_=neo4j.Result.to_df
)


# Training cyphers

In [2]:
# Reading the files
import os
os.chdir('D:\\Github_repos\\NLP_segmentation_Knowledge_Graph_cb\\Codes\\Kaggle\\19_April_thermo_results')

In [38]:
query_ex="""
MATCH (n)-[r]->(m)
WHERE r.description contains 'first order reaction'
RETURN n.name as concept_1, m.name as concept_2,
r.edge as r_edge, r.description as r_description
"""

query_ex="""
MATCH (n)-[r]->(m)
WHERE n.name CONTAINS 'homogeneous system' or m.name contains 'homogeneous system' or r.description contains 'homogeneous system' 
RETURN n.name as concept_1,
r.edge as r_edge, r.description as r_description, m.name as concept_2"""


query_ex="""MATCH (n:Concept {name: 'chemical equation'})--(m)
RETURN n.name,m.name
"""

#Find shortest path between two nodes:
query_ex="""MATCH p=shortestPath((n:Concept {name: 'isomthermal process'})-[*..15]-(m:Concept {name: 'process'}))
RETURN p

"""
#Find all nodes that do not have a specific property:
query_ex="""
MATCH (n:Concept)
WHERE (n.property) is not  NULL
RETURN n

"""

example_df=run_query_and_return_df(query_ex)
example_df

,n


In [39]:
examples = """
# What are some of the concepts directly related to "internal energy"?

MATCH (n)-[r]->(m)
WHERE n.name CONTAINS 'internal energy' AND r.edge contains 'related'
RETURN n.name as concept_1,
r.edge as r_edge, r.description as r_description, m.name as concept_2

# What is a first order reaction and associated entities?

MATCH (n)-[r]->(m)
WHERE r.description contains 'first order reaction'
RETURN n.name as concept_1, m.name as concept_2,
r.edge as r_edge, r.description as r_description


# What is a first order reaction?

MATCH (n)-[r]->(m)
WHERE r.description contains 'first order reaction'
RETURN r.description as r_description


# What can you tell me about the homogeneous systems?

MATCH (n)-[r]->(m)
WHERE n.name CONTAINS 'homogeneous system' or m.name contains 'homogeneous system' or r.description contains 'homogeneous system' 
RETURN n.name as concept_1,
r.edge as r_edge, r.description as r_description, m.name as concept_2

# What are all the nodes related to chemical reaction
MATCH (n:Concept {name: 'chemical equation'})--(m)
RETURN n.name,m.name


# Find all relationships for a specific keyword:
MATCH ()-[r:RELATIONSHIP]->()
where r.description contains 'energy'
RETURN r.edge,r.description


"""


## Chatbots to generate cypher queries

### Testing the Open AI and GoogleAI API

In [ ]:
!pip install openai

In [ ]:
pip install -q -U google-generativeai

In [50]:
import openai
from openai import OpenAI

In [51]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

In [ ]:
chat_completion=client.chat.completions.create(messages=[
    {
    "role":"user",
    "content":"Say this is a welcome party to chatgpt",
    }
],
    model="gpt-3.5-turbo"

)

In [74]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

#From Google Gemini AI

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [60]:
genai.configure(api_key=Google_API_KEY)

In [62]:
for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [63]:
model=genai.GenerativeModel('gemini-pro')

In [80]:
response=model.generate_content("what does KG guided QnA mean give a detailed response? ")#,stream=False)
to_markdown(response.text)

> **KG Guided QnA**
> 
> **Definition:**
> 
> KG Guided QnA (Knowledge Graph Guided Question Answering) is an advanced natural language processing (NLP) technique that utilizes a knowledge graph (KG) to enhance question answering systems.
> 
> **Concept:**
> 
> A knowledge graph is a structured database that represents knowledge about the world in the form of interconnected entities and relationships. It contains concepts, objects, events, and properties organized in a way that allows machines to extract and infer meaning.
> 
> KG Guided QnA incorporates the knowledge graph into the question answering process to:
> 
> * **Augment Question Understanding:** The KG provides additional context and information, enabling the QnA system to better understand the user's query.
> * **Enhance Answer Retrieval:** By matching the query to entities and relationships in the KG, the QnA system can retrieve more accurate and relevant answers.
> * **Generate Explanations:** The KG allows the QnA system to explain the reasoning behind its answers, providing users with more context and transparency.
> 
> **Process:**
> 
> KG Guided QnA typically involves the following steps:
> 
> 1. **Query Parsing:** The query is analyzed to identify entities, relationships, and keywords.
> 2. **KG Lookup:** The parsed query is matched against the knowledge graph to retrieve relevant entities and relationships.
> 3. **Answer Generation:** The KG-enhanced query is used to search for and retrieve answers from relevant sources.
> 4. **Answer Explanation:** The KG is used to provide explanations or justifications for the retrieved answers.
> 
> **Benefits of KG Guided QnA:**
> 
> * **Improved Accuracy:** The KG provides a rich source of knowledge, resulting in more precise and informative answers.
> * **Enhanced Relevance:** By leveraging the semantic relationships in the KG, the QnA system can retrieve answers that are directly related to the user's query.
> * **Explainability:** The KG assists in explaining the reasoning behind the answers, increasing user confidence and understanding.
> * **Factual Correctness:** The structured nature of the KG ensures the reliability and credibility of the answers.
> * **Domain Specialization:** KG Guided QnA can be customized to specific domains by integrating domain-specific knowledge graphs.
> 
> **Applications:**
> 
> KG Guided QnA finds applications in various domains, including:
> 
> * **Virtual Assistants:** Providing accurate and conversational answers to user questions.
> * **Chatbots:** Engaging in natural language conversations and answering customer queries.
> * **Search Engines:** Enhancing search results with KG-derived information and explanations.
> * **Healthcare:** Providing medical information and answering patient questions.
> * **Financial Services:** Providing financial insights and guidance based on KG knowledge.

In [81]:
response.prompt_feedback ##  if the response is not returned it could be because of the blockage

In [45]:
## Creating system prompt for generating cyphers for Neo4jDB
sys_prompt_1=f""" You are an AI system specializes in generating Neo4jDB Cypher queries based on example Cypher queries.
Example Neo4jDB Cypher queries are: \n {examples} \n
You will refrain from providing explanations or additional information and solely focus on generating the Neo4jDB cypher queries.
You will strictly adhere to generating Neo4jDB cypher queries based on the given examples.
However, if the context of the conversation is insufficient, you will inform the user and specify the missing context.
I repeat, if the context of the conversation is insufficient please inform the user and specify the missing context.
"""
#Do not provide any AQL queries that can't be deduced from AQL query examples. 


In [ ]:
def schoolaiChatbot(user_input):
    messages=[{"role":"system","content":sys_prompt_1}]
    messages.append({"role":"user","content":user_input})
    #change the framework to Gemini